## Creating and selecting `pl.List` and `pl.Array` columns
By the end of this lecture you will be able to:
- create `pl.List` and `pl.Array` columns (together referred to as sequence columns here)
- select sequence columns
- explode a sequence column into rows
- convert a sequence column to a `pl.Struct` or a Numpy array
- know when to use a `pl.List` or `pl.Array` column

Polars has two dtypes that can hold a sequence of values on each row. We explore their similarities and differences here.

In [ ]:
import polars as pl
import numpy as np

## Creating a sequence column
### Creating a `pl.List` column from a `list`
We create a `DataFrame` with integer, floating point and string `pl.List` columns. We set `strict=False` so that we can mix integers and floats in the input data for `floats`

In [ ]:
df_lists = pl.DataFrame(
    {
        'ints':[ 
            [0,1], 
            [2,3]
        ],
        'floats':[ 
            [0.0,1], 
            [2,3]
        ],
        'strings':[ 
            ["0","1"],
            ["2","3"]
        ]
    },
    strict=False
)
df_lists

In this case we passed `strict=False`. This is because the `lists` to initialise the `floats` column contain a mix of `floats` and `ints`. By default Polars would raise an `Exception` at these mismatched types so we set `strict=False`.

In the printed representation we see a list on each row.

In reality the data on each row of a `pl.List` column is a Polars `Series`.

We can see the underlying `Series` by selecting a row in a `pl.List` column

In [ ]:
df_lists[0,"ints"]

We can configure how many list elements are printed with a `pl.Config` setting

In [ ]:
pl.Config.set_fmt_table_cell_list_len(20)

### Creating a `pl.Array` column from a `list`
Polars also have the `pl.Array` dtype. The fundamental difference is that:
- for a `pl.List` column the sequence on each row can have different lengths
- for a `pl.Array` column the sequence on each row must have the same length

As the sequence length is predictable for the `pl.Array` dtype it may use less memory and run computation faster in some cases with large datasets.

To create a `pl.Array` dtype in `pl.DataFrame` we need to pass in the schema

In [ ]:
df_arrays = pl.DataFrame(
    {
        'ints':[ 
            [0,1], 
            [2,3]
        ],
        'floats':[ 
            [0.0,1], 
            [2,3]
        ],
        'strings':[ 
            ["0","1"],
            ["2","3"]
        ]
    },
    schema={
        "ints":pl.Array(pl.Int32,2),
        "floats":pl.Array(pl.Float32,2),
        "strings":pl.Array(pl.String,2)
    }
)
df_arrays

When we create a `pl.Array` dtype we need to pass:
- the inner dtype and
- the shape of the sequence on all rows

We can create a `pl.Array` column from a `pl.List` column with the `list.to_array` expression along with the `width` of each row

In [ ]:
(
    df_lists
    .select(
        pl.col("ints"),
        pl.col("ints").list.to_array(width=2).alias("ints_array")
    )
)

### Creating a `pl.List` column from a function
We can also create `pl.List` columns from functions. 

In this example we use `pl.int_ranges` which creates a list from a start and end point on each row. The start and end can be given as scalars or expressions

In [ ]:
(
    pl.DataFrame(
        {
            "index":[0,1,2,3]
        }
    )
    .with_columns(
        pl.int_ranges(0,pl.col("index")+1).alias("range")
    )
)

### Creating a sequence column by horiztonally-concatenating other columns
We can gather up values from other columns into a single `pl.List` column with `pl.concat_list`

In [ ]:
(
    pl.DataFrame(
        {
            "val1":[0,1,2,3],
            "val2":[10,11,12,13],
        }
    )
    .with_columns(
        pl.concat_list("val1","val2").alias("concat")
    )
)

In this case we passed `alias` to set the name of the output column. If we did not pass `alias` then the column would be called `val1` - this follows a general rule in Polars that if we create an expression from multiple columns then the name is set by the first input column listed.

We can also create a `pl.Array` column with `pl.concat_arr`

In [ ]:
(
    pl.DataFrame(
        {
            "val1":[0,1,2,3],
            "val2":[10,11,12,13],
        }
    )
    .with_columns(
        pl.concat_arr("val1","val2").alias("concat")
    )
)

### Creating higher-dimensional arrays
While a `pl.List` is 1-dimensional, a `pl.Array` can be higher dimensional when we pass a `tuple` as the `shape` parameter

In [ ]:
(
    pl.DataFrame(
        {
            "arr_1d":[[0,1]],
            "arr_2d":[[[-10.5], [0.0], [10.5]]],
        },
    schema={
        "arr_1d":pl.Array(shape=2,inner=pl.Int64),
        "arr_2d":pl.Array(shape=(3,1),inner=pl.Float64),
    }
    )
)

## Selecting sequence columns
To select multiple `pl.List` columns we must pass the column dtype

In this example we select the 64-bit integer `pl.List` column 

In [ ]:
(
    df_lists
    .select(
        pl.col(pl.List(pl.Int64))
    )
)

To select multiple `pl.Array` columns we must pass the inner dtype and length (although here only one column matches)

In [ ]:
(
    df_arrays
    .select(
        pl.col(pl.Array(pl.String,2))
    )
)

### Length of `pl.List` columns
The length of a `pl.List` column does not have to be the same on each row

In [ ]:
(
    pl.DataFrame(
        {
            'values':[ 
                [0,1], 
                [2,3,4],
                [4,5,6,7,8]
            ],
        }
    )
)

We can get the length with a `pl.List` expression. These are grouped under the `.list` namespace

In [ ]:
(
    pl.DataFrame(
        {
            'values':[ 
                [0,1], 
                [2,3,4],
                [4,5,6,7,8]
            ],
        }
    )
    .with_columns(
        pl.col("values").list.len().alias("len")
    )
)

We see more about list and array expressions in the next lecture.

## Turning sequence columns into rows
We use `explode` to expand each sequence into its own row. To illustrate this we first create a simpler `DataFrame` with an `id` column and a `values` `pl.List` column

In [ ]:
df_list = pl.DataFrame(
    {
        "id":["a","b"],
        'values':[ 
            [0,1], 
            [2,3,4]
        ],
    }
)
df_list

We can convert each list element into its own row by calling `explode` on the `DataFrame`

In [ ]:
(
    df_list
    .explode("values")
)

The syntax is the same for a `pl.Array` column.

### Using `explode` to do operations on sequence columns
In the next lecture we see how to apply expressions to `pl.List` columns. However, using `explode` is often more performant than using `list` expressions and in some cases  it is easier to write and debug operations using `explode` than using list expressions (particularly where the list expressions require the `eval` approach we see in the next lecture).

For example, here we want to create a new column that has the rank of the values in each list (with the lowest value having rank 1 etc).

We do this first by:
- doing `explode` on the list column
- doing the `rank` expression (using `over` to do it by `list`: we see more on `over` in the next section)

In [ ]:
(
    df_list
    .explode("values")
    .with_columns(
        pl.col("values").rank().over("id").cast(pl.Int32).alias("rank")
    )
)

We can then gather the data back-up into `pl.List` columns by using `group_by` on the `id` column and aggregating the `values` and `rank` by `id`

In [ ]:
(
    df_list
    .explode("values")
    .with_columns(
        pl.col("values").rank().over("id").cast(pl.Int32).alias("rank")
    )
    .group_by("id", maintain_order=True)
    .agg(
        pl.col("values"), pl.col("rank")
    )
)

## Convert a sequence column to a `pl.Struct` column
We convert a `pl.List` column to a `pl.Struct` column with `list.to_struct`.

> Be aware that data is stored in memory differently in a `pl.List` column compared to a `pl.Struct` column.
> - In a `pl.List` column each row horizontally is a `Series`
> - In a `pl.Struct` column each nested column vertically is a `Series`.
> 
> Transforming from a `pl.List` to `pl.Struct` may be memory intensive for large `DataFrames`

In this example we use `list.to_struct` to turn the `pl.List` column into `DataFrame` columns

In [ ]:
(
    pl.DataFrame(
        {
            'values':[ 
                [0,1], 
                [2,3],
                [4,5]
            ],
        }
    )
    # Convert the lists to a struct
    .with_columns(
        pl.col("values").list.to_struct().alias("value_struct")
    )
)

Similarly we can do the same for a `pl.Array` column with `arr.to_struct`.

We can then create full `DataFrame` columns from the `pl.Struct` column by calling `unnest` on the `DataFrame`

In [ ]:
(
    pl.DataFrame(
        {
            'values':[ 
                [0,1], 
                [2,3],
                [4,5]
            ],
        }
    )
    # Convert the lists to a struct
    .with_columns(
        pl.col("values").list.to_struct().alias("value_struct")
    )
    # Un-nest the struct to DataFrame columns
    .unnest("value_struct")
)

While a `pl.List` list can have a variable number of elements a `pl.Struct` has a fixed number of elements on each row. The number of struct nested columns is set by the length of the first row the `pl.List` column:
- if subsequent `pl.List` rows are shorter then these become `null` values in the `pl.Struct` nested columns
- if subsequent `pl.List` rows are longer then these are dropped from the the `pl.Struct` nested columns

Try this yourself by adding or removing values from the second row of the lists above

## Convert a sequence column to a Numpy array
A sequence column is a natural way to hold array data that we may need to conver to a two dimensional Numpy array.

### Convert a `pl.List` column to a Numpy array

Here we create a `pl.List(pl.Float64)` column that holds `embeddings` from a machine learning model (don't worry if you've never heard of embeddings, they are just one-dimensional arrays of floating point numbers)

In [ ]:
df_embeddings = (
    pl.DataFrame(
        {
            'embeddings':[ 
                [0.0,1.0], 
                [2.0,3.0],
                [4.0,5.0]
            ],
        }
    )
)
df_embeddings

If we call `to_numpy` directly on the `embeddings` column we get a **one-dimensional `object`** Numpy array where each element is an array

In [ ]:
(
    df_embeddings["embeddings"]
    .to_numpy()
)

To get the `embeddings` column as a two-dimensional Numpy array we can `explode` the column and then reshape it in Numpy

In [ ]:
(
    df_embeddings
    ["embeddings"]
    .explode()
    .to_numpy()
    .reshape(len(df_embeddings),-1)
)

### Convert a `pl.Array` column to a Numpy array
A `pl.Array` column is an even better fit for holding data that might go to/from Numpy as a `pl.Array` column has a fixed shape like a Numpy array.

The fixed-shape `pl.Array` dtype converts directly to a Numpy two-dimensional array without a reshape

In [ ]:
(
    df_embeddings
    .with_columns(
        pl.col("embeddings").list.to_array(2)
    )
    ["embeddings"]
    .to_numpy()
)

## So, should you use a `pl.List` or a `pl.Array`?
- If your rows might have variable length use `pl.List`
- If your dimension is greater than 1 use `pl.Array`
- The `pl.List` pre-dates the `pl.Array` and has more functionality in the `list` expression namespace
- I have heard people say that `pl.Array` is better performing because Polars can optimise around the fixed sequence length. However, I have not found any noticeable performance difference between them in my (limited) experiments

Overall, I recommend sticking with `pl.List` unless you find a clear performance difference for `pl.Array` in your data because there is more functionality for `pl.Array`.

To illustrate my point on performance I do a performance comparison here. The parameters are:
- `N` the number of rows (which I set to a small value here to avoid slowing down my automated testing system but which you should increase to e.g. `100_000`)
- `K` the length of each sequence
We first make a numpy `array` with random data and then a `DataFrame`

In [ ]:
N = 100
K = 1_000
arr = np.random.standard_normal((N,K))
arr_df = pl.DataFrame(arr)

We then make a `DataFrame` with a `pl.List` column and a `pl.Array` column

In [ ]:
comparison_df = (
    arr_df
    .select(
        pl.concat_list(pl.all()).alias("list"),
        pl.concat_arr(pl.all()).alias("arr"),
    )
)
comparison_df.head(2)

We then compare performance on an arithemtic operation

In [ ]:
%%timeit -n1 -r3
(
    comparison_df
    .select(
        pl.col("list").list.sum()
    )
)

In [ ]:
%%timeit -n1 -r3
(
    comparison_df
    .select(
        pl.col("arr").arr.sum()
    )
)

On my machine the performance is similar for both methods after repeated runs (let me know if you find something different!)

## Exercises
In the exercises you will develop your understanding of:
- selecting list columns
- exploding list columns
- counting occurences in a list column

### Exercise 1
We create a `DataFrame` with `pl.List` columns

In [ ]:
df_lists = pl.DataFrame(
    {
        'ints':[ 
            [0,1], 
            [2,3]
        ],
        'floats':[ 
            [0.0,1], 
            [2,3]
        ],
        'strings':[ 
            ["0","1"],
            ["2","3"]
        ]

    },
    strict=False
)
df_lists

Select the floating point list column from `df_lists`

In [ ]:
(
    df_lists
    <blank>
)

Select the floating point **and** integer list column from `df_lists`

In [ ]:
(
    df_lists
    <blank>
)

Convert the `strings` list column to a `pl.Array` column

In [ ]:
(
    df_lists
    <blank>
)

### Exercise 2
We create a `pl.List` column from the Titanic dataset by splitting the `Name` column on every whitespace

In [ ]:
csv_file = "../data/titanic.csv"
df = (
    pl.read_csv(csv_file)
    .select(
        [
            "PassengerId",
            "Pclass",
            "Name",
            pl.col("Name").str.split(" ").alias("Name_list")
        ]
    )
)
df.head(2)

Expand the `Name_list` column into separate rows

In [ ]:
(
    df
    <blank>
    .head()
)

Filter to remove rows with the titles: "Mr.","Mrs.","Miss.","Master." from the output

Find the most common names:

After filtering the titles count the occurence of each name in the `Name_list` column using `.value_counts(sort=True)`

## Solutions

### Solution to exercise 1
We create a `DataFrame` with `pl.List` columns. We use `strict=False` so that we can pass a mix of floats and integers for the `floats` column

In [ ]:
df_lists = pl.DataFrame(
    {
        'ints':[ 
            [0,1], 
            [2,3]
        ],
        'floats':[ 
            [0.0,1], 
            [2,3]
        ],
        'strings':[ 
            ["0","1"],
            ["2","3"]
        ]

    },
    strict=False
)
df_lists

Select the floating point list column from `df_lists`

In [ ]:
(
    df_lists
    .select(
        pl.col(pl.List(pl.Float64))
    )
)

Select the floating point **and** integer list column from `df_lists`

In [ ]:
(
    df_lists
    .select(
        pl.col([pl.List(pl.Float64),pl.List(pl.Int64)])
    )
)

Convert the `strings` list column to a `pl.Array` column

In [ ]:
(
    df_lists
    .select(
        pl.col("strings").list.to_array(2)
    )
)

### Solution to exercise 2
We create a `pl.List` column from the Titanic dataset by splitting the `Name` column on every whitespace

In [ ]:
csv_file = "../data/titanic.csv"
df = (
    pl.read_csv(csv_file)
    .select(
        "PassengerId",
        "Pclass",
        "Name",
        pl.col("Name").str.split(" ").alias("Name_list")
    )
)
df
# .head(2)

Expand the `Name_list` column into separate rows

In [ ]:
(
    df
    .explode("Name_list")
    .head()
)

Filter to remove rows with the titles: "Mr.","Mrs.","Miss.","Master." from the output

In [ ]:
(
    df
    .explode("Name_list")
    .filter(~pl.col("Name_list").is_in(["Mr.","Mrs.","Miss.","Master."]))
    .head()
)

Find the most common names:

After filtering the titles count the occurence of each name in the `Name_list` column using `.value_counts(sort=True)`

In [ ]:
(
    df
    .explode("Name_list")
    .filter(~pl.col("Name_list").is_in(["Mr.","Mrs.","Miss.","Master."]))
    ["Name_list"]
    .value_counts(sort=True)
    .head()
)